In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import time

In [ ]:
df = pd.read_csv('train.csv', encoding='cp949')

In [ ]:
df.columns

Index(['num', 'date_time', '전력사용량(kWh)', '기온(°C)', '풍속(m/s)', '습도(%)',
       '강수량(mm)', '일조(hr)', '비전기냉방설비운영', '태양광보유'],
      dtype='object')

In [ ]:
df['new_date_time'] = df['date_time'].str[:-2]
df['time'] = df['date_time'].str[-2:]

In [ ]:
df.select_dtypes(include="object").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122400 entries, 0 to 122399
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   date_time      122400 non-null  object
 1   new_date_time  122400 non-null  object
 2   time           122400 non-null  object
dtypes: object(3)
memory usage: 2.8+ MB


In [ ]:
df.head()

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,new_date_time,time
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,2020-06-01,00
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,2020-06-01,01
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2020-06-01,02
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,2020-06-01,03
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,2020-06-01,04


In [ ]:
df['time'] = df['time'].astype("int")

In [ ]:
X = df[['time', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '강수량(mm)', '일조(hr)', '비전기냉방설비운영', '태양광보유']].values
y = df['전력사용량(kWh)'].values

print(X)
print(y)

[[ 0.  17.6  0.8 ...  0.   0.   0. ]
 [ 1.  17.7  0.3 ...  0.   0.   0. ]
 [ 2.  17.5  0.  ...  0.   0.   0. ]
 ...
 [21.  27.3  0.  ...  0.   1.   1. ]
 [22.  27.1  0.  ...  0.   1.   1. ]
 [23.  27.1  0.  ...  0.   1.   1. ]]
[8179.056 8135.64  8107.128 ... 3572.208 3299.184 3204.576]


In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(X)

X_extended = pd.concat([pd.DataFrame(X), pd.DataFrame(cluster_labels, columns=['cluster'])], axis=1).values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_extended, y, test_size=0.2, random_state=42)

In [ ]:
scaler = RobustScaler()
X_train[:, :-1] = scaler.fit_transform(X_train[:, :-1])  # 클러스터 레이블 제외하고 정규화 (클러스터 라벨의 의미를 왜곡할 수 있음!)
X_valid[:, :-1] = scaler.transform(X_valid[:, :-1])

In [ ]:
torch.FloatTensor(X_train)

tensor([[-0.5455, -0.0213,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.9091, -0.7660,  0.0000,  ..., -1.0000,  1.0000,  2.0000],
        [ 0.7273,  0.6383,  0.0000,  ...,  0.0000,  0.0000,  2.0000],
        ...,
        [ 0.7273,  0.3830,  0.0000,  ..., -1.0000,  0.0000,  2.0000],
        [-0.8182,  0.4681,  0.0000,  ...,  0.0000,  1.0000,  2.0000],
        [ 0.1818,  0.0000,  0.0000,  ...,  0.0000,  1.0000,  2.0000]])

In [ ]:
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train).view(-1, 1)
X_valid = torch.FloatTensor(X_valid)
y_valid = torch.FloatTensor(y_valid).view(-1, 1)

In [ ]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
class MLPRegression(nn.Module):
    def __init__(self):
        super(MLPRegression, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.5)

        self.layer2 = nn.Linear(64, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.5)

        self.layer3 = nn.Linear(128, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(0.5)

        self.layer4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dropout4 = nn.Dropout(0.5)

        self.layer5 = nn.Linear(64, 32)
        self.bn5 = nn.BatchNorm1d(32)
        self.dropout5 = nn.Dropout(0.5)

        self.output_layer = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.dropout1(x)
        x = torch.relu(self.layer2(x))
        x = self.dropout2(x)
        x = torch.relu(self.layer3(x))
        x = self.dropout3(x)
        x = torch.relu(self.layer4(x))
        x = self.dropout4(x)
        x = torch.relu(self.layer5(x))
        x = self.dropout5(x)
        x = self.output_layer(x)
        return x

model = MLPRegression()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU가 사용 가능한지 확인
model = MLPRegression().to(device)  # 모델을 GPU로 이동
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

In [ ]:
early_stopping_patience = 5  # 평가 loss 개선이 없을 경우, 모델한테 줄 기회 횟수.
early_stopping_counter = 0
best_loss = float('inf')

In [ ]:
num_epochs = 50
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)  # 데이터를 GPU로 이동
        # 예측값 계산
        predictions = model(batch_X)
        # 손실 계산
        loss = criterion(predictions, batch_y)

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 역전파 및 가중치 갱신
        loss.backward()
        optimizer.step()
    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


end_time = time.time()
elapsed_time = end_time - start_time

model.eval()
with torch.no_grad():
    X_valid, y_valid = X_valid.to(device), y_valid.to(device)  # 검증 데이터도 GPU로 이동
    predictions = model(X_valid)
    loss = criterion(predictions, y_valid)
    print(f'평가 손실 Loss: {loss.item():.4f}')



Epoch [2/50], Loss: 6213443.0000
Epoch [4/50], Loss: 7546304.0000
Epoch [6/50], Loss: 6034890.0000
Epoch [8/50], Loss: 4181175.0000
Epoch [10/50], Loss: 8923010.0000
Epoch [12/50], Loss: 4052949.5000
Epoch [14/50], Loss: 3290719.0000
Epoch [16/50], Loss: 4210704.0000
Epoch [18/50], Loss: 5054782.5000
Epoch [20/50], Loss: 5250122.0000
Epoch [22/50], Loss: 2312504.0000
Epoch [24/50], Loss: 4178481.5000
Epoch [26/50], Loss: 2541492.7500
Epoch [28/50], Loss: 3390272.0000
Epoch [30/50], Loss: 8108158.5000
Epoch [32/50], Loss: 4080465.0000
Epoch [34/50], Loss: 3554674.5000
Epoch [36/50], Loss: 4723189.5000
Epoch [38/50], Loss: 3564014.7500
Epoch [40/50], Loss: 1384053.3750
Epoch [42/50], Loss: 2816755.0000
Epoch [44/50], Loss: 4112298.0000
Epoch [46/50], Loss: 2197298.2500
Epoch [48/50], Loss: 1367523.2500
Epoch [50/50], Loss: 1794891.6250
평가 손실 Loss: 3665836.5000


In [ ]:
# 결과 출력
print(f"예측 값: {predictions[:5].view(-1).tolist()}")
print(f"실제 값: {y_valid[:5].view(-1).tolist()}")

# 전체 학습 시간 출력
print(f"총 학습 시간: {elapsed_time:.2f}초")

예측 값: [2106.952392578125, 2578.763427734375, 3286.92626953125, 2371.873046875, 2443.3115234375]
실제 값: [3515.615966796875, 1801.115966796875, 2101.248046875, 8693.568359375, 1560.06005859375]
총 학습 시간: 436.52초
